In [1]:
from __future__ import print_function 
import os, sys, h5py
import numpy as np
from six.moves import cPickle
from collections import OrderedDict
from sklearn.metrics import roc_curve, auc, roc_auc_score
import matplotlib.pyplot as plt
%matplotlib inline

sys.path.append('/home/peter/Code/deepomics')
from neuralnetwork import NeuralNet, NeuralTrainer
import train as fit 
import visualize, utils

from lasagne import layers, nonlinearities, updates, objectives, init 
from lasagne.layers import get_output, get_output_shape, get_all_params
import theano.tensor as T
import theano


WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 1: TITAN X (Pascal) (CNMeM is disabled, cuDNN 5105)


In [2]:
filename = 'processed_dataset.hdf5'
data_path = '/home/peter/Code/tensorflow/data'
file_path = os.path.join(data_path, filename)
group_name = ['processed_data']
dataset = h5py.File(file_path,'r')
%time dtf = np.array(dataset['/'+group_name[0]+'/dtf'])
ltf = np.array(dataset['/'+group_name[0]+'/ltf'])
dtf_crossval = np.array(dataset['/'+group_name[0]+'/dtf_crossval'])
ltf_crossval = np.array(dataset['/'+group_name[0]+'/ltf_crossval'])

train = (dtf.transpose([0,3,1,2]), ltf)
valid = (dtf_crossval.transpose([0,3,1,2]), ltf_crossval)


CPU times: user 476 ms, sys: 1.11 s, total: 1.59 s
Wall time: 1.59 s


In [ ]:
def validation_results(nnmodel):
    def load_validation_data(file_path):
        group_name = ['valid_data']
        dataset = h5py.File(file_path,'r')
        val_dat = np.array(dataset['/'+group_name[0]+'/vs_valid'])
        val_lbl = np.array(dataset['/'+group_name[0]+'/label_valid'])
        return val_dat, val_lbl

    # load data
    file_path = os.path.join(data_path,'valideval_dataset.hdf5')
    val_dat, val_lbl = load_validation_data(file_path)

    N=140
    fragLen=330

    avg_F = np.mean(val_dat,axis=0)

    startgap = np.ceil((val_dat.shape[1] - fragLen)/14.).astype('int')
    true_lbl = np.zeros((val_dat.shape[0]*val_dat.shape[0],), dtype='float32')
    pred_lbl = np.zeros((val_dat.shape[0]*val_dat.shape[0],), dtype='float32')

    # Counter for the "true_lbl" array
    cnt_ = 0
    # Counter for the "pred_lbl" array
    cnt_u = 0

    for a in range(val_dat.shape[0]):
        if a%100 == 0:
            print(a)

        # Create batch array to send thru network
        im_eval = np.empty((N*val_dat.shape[0],3,fragLen,1), dtype='float32')

        # Count the number of traces in each batch
        cnt = 0

        for b in range(val_dat.shape[0]):

            for n in range(0, val_dat.shape[1] - fragLen, startgap):
                try:
                    im_eval[cnt,:,:,0] = np.vstack((val_dat[a,n:n+fragLen],
                                         val_dat[b,n:n+fragLen],
                                         avg_F[n:n+fragLen]))
                except:
                    from IPython.core.debugger import Tracer
                    Tracer()()

                cnt += 1

            # Keep track of the true labels
            if val_lbl[a,b] == 1:
                true_lbl[cnt_] = 1
            else:
                true_lbl[cnt_] = 0

            cnt_ += 1

        # Run batch through network
        pred_stop = nnmodel.get_feature_maps(layer='output', X=im_eval.transpose([0,3,1,2]))

        # Average output over each group of N traces
        for u in range(0, len(pred_stop), N):
            pred_lbl[cnt_u] = np.mean(pred_stop[u:u+N,0])
            cnt_u += 1       

    fpr, tpr, thresholds = roc_curve(true_lbl, pred_lbl)
    return auc(fpr, tpr)
    

# original model

In [ ]:
def build_model(shape, num_labels):

    # get model
    input_var = T.tensor4('inputs')
    target_var = T.dmatrix('targets')

    net = {}
    net['input'] = layers.InputLayer(input_var=input_var, shape=shape)
    net['conv1'] = layers.Conv2DLayer(net['input'], num_filters=18, filter_size=(2, 5), stride=(1, 1),    # 1000
                                        W=init.HeNormal(), b=init.Constant(0.1), 
                                      nonlinearity=nonlinearities.tanh, pad='valid')

    net['conv2'] = layers.Conv2DLayer(net['conv1'], num_filters=40, filter_size=(2, 5), stride=(1, 1), # 18
                                        W=init.HeNormal(), b=init.Constant(0.1), 
                                      nonlinearity=nonlinearities.tanh, pad='valid')
    net['conv2_pool'] = layers.MaxPool2DLayer(net['conv2'], pool_size=(1, 10), stride=(1, 10)) # 25

    net['conv3'] = layers.Conv2DLayer(net['conv2_pool'], num_filters=15, filter_size=(1, 1), stride=(1, 1), # 18
                                        W=init.HeNormal(), b=init.Constant(0.1), 
                                      nonlinearity=nonlinearities.tanh, pad='valid')

    net['dense1'] = layers.DenseLayer(net['conv3'], num_units=100, W=init.HeNormal(), 
                                     b=init.Constant(), nonlinearity=nonlinearities.rectify)
    
    net['output'] = layers.DenseLayer(net['dense1'], num_units=num_labels, W=init.HeNormal(), 
                                     b=init.Constant(), nonlinearity=nonlinearities.sigmoid)
    
    # optimization parameters
    optimization = {"objective": "binary",
                    "optimizer": "adam",
                    "learning_rate": 0.001, 
                    "l2": 1e-6
                    }

    return net, input_var, target_var, optimization

# build network
shape = (None, train[0].shape[1], train[0].shape[2], train[0].shape[3])
num_labels = train[1].shape[1]
net, input_var, target_var, optimization = build_model(shape, num_labels)

# build neural network class
nnmodel = NeuralNet(net, input_var, target_var)
#nnmodel.inspect_layers()

# set output file paths
resultspath = utils.make_directory(data_path, 'results')
resultspath = utils.make_directory(resultspath, 'deepomics')
output_name = 'original_model'
filepath = os.path.join(resultspath, output_name)
nntrainer = NeuralTrainer(nnmodel, optimization, save='best', filepath=filepath)

In [ ]:
# train model
fit.train_minibatch(nntrainer, data={'train': train, 'valid': valid}, 
                              batch_size=1000, num_epochs=500, patience=10, verbose=1)


In [ ]:
nntrainer.set_best_parameters()

# test model
nntrainer.test_model(valid, batch_size=100, name="test")

In [ ]:
# check official validation error
auc = validation_results(nnmodel)
print('validation AUC: ' + str(auc))

# original model w/ batch norm and dropout and relu

In [ ]:
def build_model(shape, num_labels):

    # get model
    input_var = T.tensor4('inputs')
    target_var = T.dmatrix('targets')

    net = {}
    net['input'] = layers.InputLayer(input_var=input_var, shape=shape)
    net['conv1'] = layers.Conv2DLayer(net['input'], num_filters=18, filter_size=(2, 5), stride=(1, 1),    # 1000
                                        W=init.HeNormal(), b=None, nonlinearity=None, pad='valid')
    net['conv1_norm'] = layers.BatchNormLayer(net['conv1'])
    net['conv1_active'] = layers.NonlinearityLayer(net['conv1_norm'], nonlinearity=nonlinearities.rectify)
    net['conv1_dropout'] = layers.DropoutLayer(net['conv1_active'], p=0.1)

    net['conv2'] = layers.Conv2DLayer(net['conv1_dropout'], num_filters=40, filter_size=(2, 5), stride=(1, 1), # 18
                                        W=init.HeNormal(), b=None, nonlinearity=None, pad='valid')
    net['conv2_norm'] = layers.BatchNormLayer(net['conv2'])
    net['conv2_active'] = layers.NonlinearityLayer(net['conv2_norm'], nonlinearity=nonlinearities.rectify)
    net['conv2_pool'] = layers.MaxPool2DLayer(net['conv2_active'], pool_size=(1, 10), stride=(1, 10)) # 25
    net['conv2_dropout'] = layers.DropoutLayer(net['conv2_pool'], p=0.1)

    net['conv3'] = layers.Conv2DLayer(net['conv2_dropout'], num_filters=15, filter_size=(1, 1), stride=(1, 1), # 18
                                        W=init.HeNormal(), b=None, nonlinearity=None, pad='valid')
    net['conv3_norm'] = layers.BatchNormLayer(net['conv3'])
    net['conv3_active'] = layers.NonlinearityLayer(net['conv3_norm'], nonlinearity=nonlinearities.rectify)
    net['conv3_dropout'] = layers.DropoutLayer(net['conv3_active'], p=0.1)

    net['dense1'] = layers.DenseLayer(net['conv3_dropout'], num_units=100, W=init.HeNormal(), 
                                     b=init.Constant(), nonlinearity=None)
    net['dense1_active'] = layers.NonlinearityLayer(net['dense1'], nonlinearity=nonlinearities.rectify)
    net['dense1_dropout'] = layers.DropoutLayer(net['dense1_active'], p=0.3)
    
    net['dense2'] = layers.DenseLayer(net['dense1_dropout'], num_units=num_labels, W=init.HeNormal(), 
                                     b=init.Constant(), nonlinearity=None)
    net['output'] = layers.NonlinearityLayer(net['dense2'], nonlinearity=nonlinearities.sigmoid)
    
    # optimization parameters
    optimization = {"objective": "binary",
                    "optimizer": "adam",
                    "learning_rate": 0.001, 
                    "l2": 1e-6
                    }

    return net, input_var, target_var, optimization

# build network
shape = (None, train[0].shape[1], train[0].shape[2], train[0].shape[3])
num_labels = train[1].shape[1]
net, input_var, target_var, optimization = build_model(shape, num_labels)

# build neural network class
nnmodel = NeuralNet(net, input_var, target_var)
#nnmodel.inspect_layers()

# set output file paths
resultspath = utils.make_directory(data_path, 'results')
resultspath = utils.make_directory(resultspath, 'deepomics')
output_name = 'original_model_bells'
filepath = os.path.join(resultspath, output_name)
nntrainer = NeuralTrainer(nnmodel, optimization, save='best', filepath=filepath)

In [ ]:
# train model
fit.train_minibatch(nntrainer, data={'train': train, 'valid': valid}, 
                              batch_size=1000, num_epochs=500, patience=10, verbose=1)


In [ ]:
nntrainer.set_best_parameters()

# test model
nntrainer.test_model(valid, batch_size=100, name="test")

In [ ]:
# check official validation error
auc = validation_results(nnmodel)
print('validation AUC: ' + str(auc))

# modified network (wider + batch norm + dropout + prelu)

In [ ]:
def build_model(shape, num_labels):

    # get model
    input_var = T.tensor4('inputs')
    target_var = T.dmatrix('targets')

    net = {}
    net['input'] = layers.InputLayer(input_var=input_var, shape=shape)
    net['conv1'] = layers.Conv2DLayer(net['input'], num_filters=32, filter_size=(2, 5), stride=(1, 1),    # 1000
                                        W=init.HeNormal(), b=None, nonlinearity=None, pad='valid')
    net['conv1_norm'] = layers.BatchNormLayer(net['conv1'])
    net['conv1_active'] = layers.ParametricRectifierLayer(net['conv1_norm'], alpha=init.Constant(0.25))
    net['conv1_dropout'] = layers.DropoutLayer(net['conv1_active'], p=0.1)

    net['conv2'] = layers.Conv2DLayer(net['conv1_dropout'], num_filters=64, filter_size=(2, 5), stride=(1, 1), # 18
                                        W=init.HeNormal(), b=None, nonlinearity=None, pad='valid')
    net['conv2_norm'] = layers.BatchNormLayer(net['conv2'])
    net['conv2_active'] = layers.ParametricRectifierLayer(net['conv2_norm'], alpha=init.Constant(0.25))
    net['conv2_pool'] = layers.MaxPool2DLayer(net['conv2_active'], pool_size=(1, 10), stride=(1, 10)) # 25
    net['conv2_dropout'] = layers.DropoutLayer(net['conv2_pool'], p=0.1)

    net['conv3'] = layers.Conv2DLayer(net['conv2_dropout'], num_filters=128, filter_size=(1, 1), stride=(1, 1), # 18
                                        W=init.HeNormal(), b=None, nonlinearity=None, pad='valid')
    net['conv3_norm'] = layers.BatchNormLayer(net['conv3'])
    net['conv3_active'] = layers.ParametricRectifierLayer(net['conv3_norm'], alpha=init.Constant(0.25))
    net['conv3_dropout'] = layers.DropoutLayer(net['conv3_active'], p=0.1)

    net['dense1'] = layers.DenseLayer(net['conv3_dropout'], num_units=256, W=init.HeNormal(), 
                                     b=init.Constant(0.1), nonlinearity=None)
    net['dense1_active'] = layers.ParametricRectifierLayer(net['dense1'], alpha=init.Constant(0.25))
    net['dense1_dropout'] = layers.DropoutLayer(net['dense1_active'], p=0.3)
    
    net['dense2'] = layers.DenseLayer(net['dense1_dropout'], num_units=num_labels, W=init.HeNormal(), 
                                     b=init.Constant(), nonlinearity=None)
    net['output'] = layers.NonlinearityLayer(net['dense2'], nonlinearity=nonlinearities.sigmoid)
    
    # optimization parameters
    optimization = {"objective": "binary",
                    "optimizer": "adam",
                    "learning_rate": 0.001, 
                    "l2": 1e-6
                    }

    return net, input_var, target_var, optimization

# build network
shape = (None, train[0].shape[1], train[0].shape[2], train[0].shape[3])
num_labels = train[1].shape[1]
net, input_var, target_var, optimization = build_model(shape, num_labels)

# build neural network class
nnmodel = NeuralNet(net, input_var, target_var)
#nnmodel.inspect_layers()

# set output file paths
resultspath = utils.make_directory(data_path, 'results')
resultspath = utils.make_directory(resultspath, 'deepomics')
output_name = 'original_model_bells_whistles'
filepath = os.path.join(resultspath, output_name)
nntrainer = NeuralTrainer(nnmodel, optimization, save='best', filepath=filepath)

In [ ]:
# train model
fit.train_minibatch(nntrainer, data={'train': train, 'valid': valid}, 
                              batch_size=100, num_epochs=20, patience=5, verbose=1)
# train model
nnmodel = NeuralNet(net, input_var, target_var)
nntrainer = NeuralTrainer(nnmodel, optimization, save='best', filepath=filepath)
nntrainer.set_best_parameters()
fit.train_minibatch(nntrainer, data={'train': train, 'valid': valid}, 
                              batch_size=500, num_epochs=20, patience=5, verbose=1)

nnmodel = NeuralNet(net, input_var, target_var)
nntrainer = NeuralTrainer(nnmodel, optimization, save='best', filepath=filepath)
nntrainer.set_best_parameters()
fit.train_minibatch(nntrainer, data={'train': train, 'valid': valid}, 
                              batch_size=1000, num_epochs=10, patience=5, verbose=1)

nnmodel = NeuralNet(net, input_var, target_var)
nntrainer = NeuralTrainer(nnmodel, optimization, save='best', filepath=filepath)
nntrainer.set_best_parameters()
fit.train_minibatch(nntrainer, data={'train': train, 'valid': valid}, 
                              batch_size=1500, num_epochs=10, patience=5, verbose=1)

nnmodel = NeuralNet(net, input_var, target_var)
nntrainer = NeuralTrainer(nnmodel, optimization, save='best', filepath=filepath)
nntrainer.set_best_parameters()
fit.train_minibatch(nntrainer, data={'train': train, 'valid': valid}, 
                              batch_size=2000, num_epochs=10, patience=5, verbose=1)


In [ ]:
nntrainer.set_best_parameters()

# test model
nntrainer.test_model(valid, batch_size=100, name="test")

In [ ]:
# check official validation error
auc = validation_results(nnmodel)
print('validation AUC: ' + str(auc))

# normal residual model

In [ ]:
def build_model(shape, num_labels):
    def residual_block(net, last_layer, name, filter_size, nonlinearity=nonlinearities.rectify):
        # original residual unit
        shape = layers.get_output_shape(net[last_layer])
        num_filters = shape[1]

        net[name+'_1resid'] = layers.Conv2DLayer(net[last_layer], num_filters=num_filters, filter_size=filter_size, stride=(1, 1),    # 1000
                         W=init.HeNormal(), b=None, nonlinearity=None, pad='same')
        net[name+'_1resid_norm'] = layers.BatchNormLayer(net[name+'_1resid'])
        net[name+'_1resid_active'] = layers.NonlinearityLayer(net[name+'_1resid_norm'], nonlinearity=nonlinearity)

        net[name+'_1resid_dropout'] = layers.DropoutLayer(net[name+'_1resid_active'], p=0.1)

        # bottleneck residual layer
        net[name+'_2resid'] = layers.Conv2DLayer(net[name+'_1resid_dropout'], num_filters=num_filters, filter_size=filter_size, stride=(1, 1),    # 1000
                         W=init.HeNormal(), b=None, nonlinearity=None, pad='same')
        net[name+'_2resid_norm'] = layers.BatchNormLayer(net[name+'_2resid'])

        # combine input with residuals
        net[name+'_residual'] = layers.ElemwiseSumLayer([net[last_layer], net[name+'_2resid_norm']])
        net[name+'_resid'] = layers.NonlinearityLayer(net[name+'_residual'], nonlinearity=nonlinearity)

        return net

    def residual_block2(net, last_layer, name, nonlinearity=nonlinearities.rectify):
        # original residual unit
        shape = layers.get_output_shape(net[last_layer])
        num_units = shape[1]

        net[name+'_1resid'] = layers.DenseLayer(net[last_layer], num_units=num_units, W=init.HeNormal(), b=None, nonlinearity=None)
        net[name+'_1resid_norm'] = layers.BatchNormLayer(net[name+'_1resid'])
        net[name+'_1resid_active'] = layers.NonlinearityLayer(net[name+'_1resid_norm'], nonlinearity=nonlinearity)

        net[name+'_1resid_dropout'] = layers.DropoutLayer(net[name+'_1resid_active'], p=0.1)

        # bottleneck residual layer
        net[name+'_2resid'] = layers.DenseLayer(net[name+'_1resid_dropout'], num_units=num_units, W=init.HeNormal(), b=None, nonlinearity=None)
        net[name+'_2resid_norm'] = layers.BatchNormLayer(net[name+'_2resid'])

        # combine input with residuals
        net[name+'_residual'] = layers.ElemwiseSumLayer([net[last_layer], net[name+'_2resid_norm']])
        net[name+'_resid'] = layers.NonlinearityLayer(net[name+'_residual'], nonlinearity=nonlinearity)

        return net

    # get model
    input_var = T.tensor4('inputs')
    target_var = T.dmatrix('targets')

    net = {}
    net['input'] = layers.InputLayer(input_var=input_var, shape=shape)
    net['conv1'] = layers.Conv2DLayer(net['input'], num_filters=18, filter_size=(2, 5), stride=(1, 1),    # 1000
                                        W=init.HeNormal(), b=None, nonlinearity=None, pad='valid')
    net['conv1_norm'] = layers.BatchNormLayer(net['conv1'])
    net['conv1_active'] = layers.NonlinearityLayer(net['conv1_norm'], nonlinearity=nonlinearities.leaky_rectify)
    net['conv1_dropout1'] = layers.DropoutLayer(net['conv1_active'], p=0.1)
    net = residual_block(net, 'conv1_dropout1', 'conv1_2', filter_size=(3,5), nonlinearity=nonlinearities.leaky_rectify)
    net['conv1_dropout'] = layers.DropoutLayer(net['conv1_2_resid'], p=0.1)

    net['conv2'] = layers.Conv2DLayer(net['conv1_dropout'], num_filters=40, filter_size=(2, 5), stride=(1, 1), # 18
                                        W=init.HeNormal(), b=None, nonlinearity=None, pad='valid')
    net['conv2_norm'] = layers.BatchNormLayer(net['conv2'])
    net['conv2_active'] = layers.NonlinearityLayer(net['conv2_norm'], nonlinearity=nonlinearities.leaky_rectify)
    net['conv2_dropout1'] = layers.DropoutLayer(net['conv2_active'], p=0.1)
    net = residual_block(net, 'conv2_dropout1', 'conv2_2', filter_size=(1,5), nonlinearity=nonlinearities.leaky_rectify)
    net['conv2_pool'] = layers.MaxPool2DLayer(net['conv2_2_resid'], pool_size=(1, 10), stride=(1, 10)) # 25
    net['conv2_dropout'] = layers.DropoutLayer(net['conv2_pool'], p=0.1)

    net['conv3'] = layers.Conv2DLayer(net['conv2_dropout'], num_filters=15, filter_size=(1, 1), stride=(1, 1), # 18
                                        W=init.HeNormal(), b=None, nonlinearity=None, pad='valid')
    net['conv3_norm'] = layers.BatchNormLayer(net['conv3'])
    net['conv3_active'] = layers.NonlinearityLayer(net['conv3_norm'], nonlinearity=nonlinearities.leaky_rectify)
    net['conv3_dropout'] = layers.DropoutLayer(net['conv3_active'], p=0.1)

    net['dense1'] = layers.DenseLayer(net['conv3_dropout'], num_units=100, W=init.HeNormal(), 
                                     b=None, nonlinearity=None)
    net['dense1_norm'] = layers.BatchNormLayer(net['dense1'])    
    net['dense1_active'] = layers.NonlinearityLayer(net['dense1_norm'], nonlinearity=nonlinearities.leaky_rectify)
    net['dense1_dropout1'] = layers.DropoutLayer(net['dense1_active'], p=0.1)
    net = residual_block2(net, 'dense1_dropout1', 'dense1_2', nonlinearity=nonlinearities.leaky_rectify)
    net['dense1_dropout'] = layers.DropoutLayer(net['dense1_2_resid'], p=0.3)
    
    net['dense2'] = layers.DenseLayer(net['dense1_dropout'], num_units=num_labels, W=init.HeNormal(), 
                                     b=init.Constant(), nonlinearity=None)
    net['output'] = layers.NonlinearityLayer(net['dense2'], nonlinearity=nonlinearities.sigmoid)
    
    # optimization parameters
    optimization = {"objective": "binary",
                    "optimizer": "adam",
                    "learning_rate": 0.001, 
                    "l2": 1e-6
                    }

    return net, input_var, target_var, optimization

# build network
shape = (None, train[0].shape[1], train[0].shape[2], train[0].shape[3])
num_labels = train[1].shape[1]
net, input_var, target_var, optimization = build_model(shape, num_labels)

# build neural network class
nnmodel = NeuralNet(net, input_var, target_var)
#nnmodel.inspect_layers()

# set output file paths
resultspath = utils.make_directory(data_path, 'results')
resultspath = utils.make_directory(resultspath, 'deepomics')
output_name = 'original_model_residual'
filepath = os.path.join(resultspath, output_name)
nntrainer = NeuralTrainer(nnmodel, optimization, save='best', filepath=filepath)

In [ ]:
# train model
nntrainer.set_best_parameters()
fit.train_minibatch(nntrainer, data={'train': train, 'valid': valid}, 
                              batch_size=100, num_epochs=20, patience=5, verbose=1)
# train model
nnmodel = NeuralNet(net, input_var, target_var)
nntrainer = NeuralTrainer(nnmodel, optimization, save='best', filepath=filepath)
nntrainer.set_best_parameters()
fit.train_minibatch(nntrainer, data={'train': train, 'valid': valid}, 
                              batch_size=500, num_epochs=20, patience=5, verbose=1)

nnmodel = NeuralNet(net, input_var, target_var)
nntrainer = NeuralTrainer(nnmodel, optimization, save='best', filepath=filepath)
nntrainer.set_best_parameters()
fit.train_minibatch(nntrainer, data={'train': train, 'valid': valid}, 
                              batch_size=1000, num_epochs=10, patience=5, verbose=1)

nnmodel = NeuralNet(net, input_var, target_var)
nntrainer = NeuralTrainer(nnmodel, optimization, save='best', filepath=filepath)
nntrainer.set_best_parameters()
fit.train_minibatch(nntrainer, data={'train': train, 'valid': valid}, 
                              batch_size=1500, num_epochs=10, patience=5, verbose=1)

nnmodel = NeuralNet(net, input_var, target_var)
nntrainer = NeuralTrainer(nnmodel, optimization, save='best', filepath=filepath)
nntrainer.set_best_parameters()
fit.train_minibatch(nntrainer, data={'train': train, 'valid': valid}, 
                              batch_size=2000, num_epochs=10, patience=5, verbose=1)


In [ ]:
nntrainer.set_best_parameters()

# test model
nntrainer.test_model(valid, batch_size=100, name="test")

In [ ]:
# check official validation error
auc = validation_results(nnmodel)
print('validation AUC: ' + str(auc))

# residual model (modified)

In [ ]:
def build_model(shape, num_labels):
    def residual_block(net, last_layer, name, filter_size, nonlinearity=nonlinearities.rectify):
        # original residual unit
        shape = layers.get_output_shape(net[last_layer])
        num_filters = shape[1]

        net[name+'_1resid'] = layers.Conv2DLayer(net[last_layer], num_filters=num_filters, filter_size=filter_size, stride=(1, 1),    # 1000
                         W=init.HeNormal(), b=None, nonlinearity=None, pad='same')
        net[name+'_1resid_norm'] = layers.BatchNormLayer(net[name+'_1resid'])
        net[name+'_1resid_active'] = layers.ParametricRectifierLayer(net[name+'_1resid_norm'], alpha=init.Constant(0.25))
            
        net[name+'_1resid_dropout'] = layers.DropoutLayer(net[name+'_1resid_active'], p=0.1)

        # bottleneck residual layer
        net[name+'_2resid'] = layers.Conv2DLayer(net[name+'_1resid_dropout'], num_filters=num_filters, filter_size=filter_size, stride=(1, 1),    # 1000
                         W=init.HeNormal(), b=None, nonlinearity=None, pad='same')
        net[name+'_2resid_norm'] = layers.BatchNormLayer(net[name+'_2resid'])

        # combine input with residuals
        net[name+'_residual'] = layers.ElemwiseSumLayer([net[last_layer], net[name+'_2resid_norm']])
        net[name+'_resid'] = layers.ParametricRectifierLayer(net[name+'_residual'], alpha=init.Constant(0.25))

        return net
    
    def residual_block2(net, last_layer, name, nonlinearity=nonlinearities.rectify):
        # original residual unit
        shape = layers.get_output_shape(net[last_layer])
        num_units = shape[1]

        net[name+'_1resid'] = layers.DenseLayer(net[last_layer], num_units=num_units, W=init.HeNormal(), b=None, nonlinearity=None)
        net[name+'_1resid_norm'] = layers.BatchNormLayer(net[name+'_1resid'])
        net[name+'_1resid_active'] = layers.ParametricRectifierLayer(net[name+'_1resid_norm'], alpha=init.Constant(0.25))

        net[name+'_1resid_dropout'] = layers.DropoutLayer(net[name+'_1resid_active'], p=0.2)

        # bottleneck residual layer
        net[name+'_2resid'] = layers.DenseLayer(net[name+'_1resid_dropout'], num_units=num_units, W=init.HeNormal(), b=None, nonlinearity=None)
        net[name+'_2resid_norm'] = layers.BatchNormLayer(net[name+'_2resid'])

        # combine input with residuals
        net[name+'_residual'] = layers.ElemwiseSumLayer([net[last_layer], net[name+'_2resid_norm']])
        net[name+'_resid'] = layers.ParametricRectifierLayer(net[name+'_residual'], alpha=init.Constant(0.25))

        return net

    # get model
    input_var = T.tensor4('inputs')
    target_var = T.dmatrix('targets')

    net = {}
    net['input'] = layers.InputLayer(input_var=input_var, shape=shape)
    net['conv1'] = layers.Conv2DLayer(net['input'], num_filters=32, filter_size=(2, 5), stride=(1, 1),    # 325
                                        W=init.HeNormal(), b=None, nonlinearity=None, pad='valid')
    net['conv1_norm'] = layers.BatchNormLayer(net['conv1'])
    net['conv1_active'] = layers.ParametricRectifierLayer(net['conv1_norm'], alpha=init.Constant(0.25))
    net['conv1_dropout1'] = layers.DropoutLayer(net['conv1_active'], p=0.1)
    net = residual_block(net, 'conv1_dropout1', 'conv1_2', filter_size=(3, 5), nonlinearity=nonlinearities.leaky_rectify)
    net['conv1_dropout'] = layers.DropoutLayer(net['conv1_2_resid'], p=0.2)

    net['conv2'] = layers.Conv2DLayer(net['conv1_dropout'], num_filters=64, filter_size=(2, 5), stride=(1, 1), # 60
                                        W=init.HeNormal(), b=None, nonlinearity=None, pad='valid')
    net['conv2_norm'] = layers.BatchNormLayer(net['conv2'])
    net['conv2_active'] = layers.ParametricRectifierLayer(net['conv2_norm'], alpha=init.Constant(0.25))
    net['conv2_dropout1'] = layers.DropoutLayer(net['conv2_active'], p=0.1)
    net = residual_block(net, 'conv2_dropout1', 'conv2_2', filter_size=(1, 5), nonlinearity=nonlinearities.leaky_rectify)
    net['conv2_pool'] = layers.MaxPool2DLayer(net['conv2_2_resid'], pool_size=(1, 10), stride=(1, 10)) # 12
    net['conv2_dropout'] = layers.DropoutLayer(net['conv2_pool'], p=0.2)

    net['conv3'] = layers.Conv2DLayer(net['conv2_dropout'], num_filters=128, filter_size=(1, 1), stride=(1, 1), # 9
                                        W=init.HeNormal(), b=None, nonlinearity=None, pad='valid')
    net['conv3_norm'] = layers.BatchNormLayer(net['conv3'])
    net['conv3_active'] = layers.ParametricRectifierLayer(net['conv3_norm'], alpha=init.Constant(0.25))
    net['conv3_dropout'] = layers.DropoutLayer(net['conv3_active'], p=0.2)
    
    net['dense1'] = layers.DenseLayer(net['conv3_dropout'], num_units=256, W=init.HeNormal(), 
                                     b=None, nonlinearity=None)
    net['dense1_norm'] = layers.BatchNormLayer(net['dense1'])    
    net['dense1_active'] = layers.ParametricRectifierLayer(net['dense1_norm'], alpha=init.Constant(0.25))
    net['dense1_dropout1'] = layers.DropoutLayer(net['dense1_active'], p=0.2)
    net = residual_block2(net, 'dense1_dropout1', 'dense1_2', nonlinearity=nonlinearities.leaky_rectify)
    net['dense1_dropout'] = layers.DropoutLayer(net['dense1_2_resid'], p=0.5)
        
    net['dense2'] = layers.DenseLayer(net['dense1_dropout'], num_units=num_labels, W=init.HeNormal(), 
                                     b=init.Constant(), nonlinearity=None)
    net['output'] = layers.NonlinearityLayer(net['dense2'], nonlinearity=nonlinearities.sigmoid)
        
    # optimization parameters
    optimization = {"objective": "binary",
                    "optimizer": "adam",
                    "learning_rate": 0.001, 
                    "l2": 1e-6
                    }

    return net, input_var, target_var, optimization

# build network
shape = (None, train[0].shape[1], train[0].shape[2], train[0].shape[3])
num_labels = train[1].shape[1]
net, input_var, target_var, optimization = build_model(shape, num_labels)

# build neural network class
nnmodel = NeuralNet(net, input_var, target_var)
#nnmodel.inspect_layers()

# set output file paths
resultspath = utils.make_directory(data_path, 'results')
resultspath = utils.make_directory(resultspath, 'deepomics')
output_name = '1D_model_resid'
filepath = os.path.join(resultspath, output_name)
nntrainer = NeuralTrainer(nnmodel, optimization, save='best', filepath=filepath)

compiling model


/home/peter/anaconda2/lib/python2.7/site-packages/lasagne/layers/pool.py:267: UserWarning: DEPRECATION: the 'ds' parameter is not going to exist anymore as it is going to be replaced by the parameter 'ws'.
  mode=self.mode,
/home/peter/anaconda2/lib/python2.7/site-packages/lasagne/layers/pool.py:267: UserWarning: DEPRECATION: the 'st' parameter is not going to exist anymore as it is going to be replaced by the parameter 'stride'.
  mode=self.mode,
/home/peter/anaconda2/lib/python2.7/site-packages/lasagne/layers/pool.py:267: UserWarning: DEPRECATION: the 'padding' parameter is not going to exist anymore as it is going to be replaced by the parameter 'pad'.
  mode=self.mode,


In [ ]:
# train model
#nntrainer.set_best_parameters()
fit.train_minibatch(nntrainer, data={'train': train, 'valid': valid}, 
                              batch_size=100, num_epochs=50, patience=10, verbose=1)
# train model
nnmodel = NeuralNet(net, input_var, target_var)
nntrainer = NeuralTrainer(nnmodel, optimization, save='best', filepath=filepath)
nntrainer.set_best_parameters()
fit.train_minibatch(nntrainer, data={'train': train, 'valid': valid}, 
                              batch_size=500, num_epochs=50, patience=10, verbose=1)

Epoch 1 out of 50 
[===                           ] 11.4% -- time=519s -- loss=0.42606 -- accuracy=83.90%  

In [ ]:
nnmodel = NeuralNet(net, input_var, target_var)
nntrainer = NeuralTrainer(nnmodel, optimization, save='best', filepath=filepath)
nntrainer.set_best_parameters()
fit.train_minibatch(nntrainer, data={'train': train, 'valid': valid}, 
                              batch_size=1000, num_epochs=50, patience=10, verbose=1)

nnmodel = NeuralNet(net, input_var, target_var)
nntrainer = NeuralTrainer(nnmodel, optimization, save='best', filepath=filepath)
nntrainer.set_best_parameters()
fit.train_minibatch(nntrainer, data={'train': train, 'valid': valid}, 
                              batch_size=1500, num_epochs=50, patience=10, verbose=1)

nnmodel = NeuralNet(net, input_var, target_var)
nntrainer = NeuralTrainer(nnmodel, optimization, save='best', filepath=filepath)
nntrainer.set_best_parameters()
fit.train_minibatch(nntrainer, data={'train': train, 'valid': valid}, 
                              batch_size=2000, num_epochs=50, patience=10, verbose=1)

In [ ]:
nntrainer.set_best_parameters()
fit.train_minibatch(nntrainer, data={'train': train, 'valid': valid}, 
                              batch_size=100, num_epochs=50, patience=10, verbose=1)
# train model
nnmodel = NeuralNet(net, input_var, target_var)
nntrainer = NeuralTrainer(nnmodel, optimization, save='best', filepath=filepath)
nntrainer.set_best_parameters()
fit.train_minibatch(nntrainer, data={'train': train, 'valid': valid}, 
                              batch_size=500, num_epochs=50, patience=10, verbose=1)

nnmodel = NeuralNet(net, input_var, target_var)
nntrainer = NeuralTrainer(nnmodel, optimization, save='best', filepath=filepath)
nntrainer.set_best_parameters()
fit.train_minibatch(nntrainer, data={'train': train, 'valid': valid}, 
                              batch_size=1000, num_epochs=50, patience=10, verbose=1)

nnmodel = NeuralNet(net, input_var, target_var)
nntrainer = NeuralTrainer(nnmodel, optimization, save='best', filepath=filepath)
nntrainer.set_best_parameters()
fit.train_minibatch(nntrainer, data={'train': train, 'valid': valid}, 
                              batch_size=1500, num_epochs=50, patience=10, verbose=1)

nnmodel = NeuralNet(net, input_var, target_var)
nntrainer = NeuralTrainer(nnmodel, optimization, save='best', filepath=filepath)
nntrainer.set_best_parameters()
fit.train_minibatch(nntrainer, data={'train': train, 'valid': valid}, 
                              batch_size=2000, num_epochs=50, patience=10, verbose=1)


nnmodel = NeuralNet(net, input_var, target_var)
nntrainer = NeuralTrainer(nnmodel, optimization, save='best', filepath=filepath)
nntrainer.set_best_parameters()
fit.train_minibatch(nntrainer, data={'train': train, 'valid': valid}, 
                              batch_size=2500, num_epochs=50, patience=10, verbose=1)

In [ ]:
nntrainer.set_best_parameters()

# test model
nntrainer.test_model(valid, batch_size=100, name="test")

In [ ]:

def validation_score(nntrainer, data_path):

    filename = 'valideval_dataset.hdf5'
    dataset = h5py.File(os.path.join(data_path,filename),'r')
    group_name = ['valid_data']
    val_dat = np.array(dataset['/'+group_name[0]+'/vs_valid'])
    val_lbl = np.array(dataset['/'+group_name[0]+'/label_valid'])

    fragLen = 330
    N = 14
    avg_F = np.mean(val_dat,axis=0)

    startgap = np.ceil(float(val_dat.shape[1] - fragLen)/N).astype('int')
    true_lbl = np.zeros((val_dat.shape[0]*val_dat.shape[0],), dtype='float32')
    pred_lbl = np.zeros((val_dat.shape[0]*val_dat.shape[0],), dtype='float32')

    # Counter for the "true_lbl" array
    cnt_ = 0
    # Counter for the "pred_lbl" array
    cnt_u = 0
    for a in range(val_dat.shape[0]):
        if a%100 == 0:
            print('\r' + 'X'*(a//100))

        # Create batch array to send thru network
        im_eval = np.empty((N*val_dat.shape[0],3,fragLen,1), dtype='float32')

        # Count the number of traces in each batch
        cnt = 0

        for b in range(val_dat.shape[0]):

            for n in range(0, val_dat.shape[1] - fragLen, startgap):
                try:
                    im_eval[cnt,:,:,0] = np.vstack((val_dat[a,n:n+fragLen],
                                         val_dat[b,n:n+fragLen],
                                         avg_F[n:n+fragLen]))
                except:
                    from IPython.core.debugger import Tracer
                    Tracer()()

                cnt += 1

            # Keep track of the true labels
            if val_lbl[a,b] == 1:
                true_lbl[cnt_] = 1
            else:
                true_lbl[cnt_] = 0

            cnt_ += 1

        # Run batch through network
        pred_stop = nntrainer.nnmodel.get_feature_maps(layer='output', X=im_eval.transpose([0,3,1,2]))[:,0]
        # Average output over each group of N traces
        for u in range(0, len(pred_stop), N):
            pred_lbl[cnt_u] = np.mean(pred_stop[u:u+N])
            cnt_u += 1        


    from sklearn.metrics import roc_curve, auc, precision_recall_curve, accuracy_score, roc_auc_score
    fpr, tpr, thresholds = roc_curve(true_lbl, pred_lbl)
    wrk = auc(fpr, tpr)
    print(wrk)
    return wrk

validation_score(nntrainer, data_path)

# 1D models

# 1D architecture

In [ ]:
def build_model(shape, num_labels):
    def residual_block(net, last_layer, name, filter_size, nonlinearity=nonlinearities.rectify):
        # original residual unit
        shape = layers.get_output_shape(net[last_layer])
        num_filters = shape[1]

        net[name+'_1resid'] = layers.Conv2DLayer(net[last_layer], num_filters=num_filters, filter_size=filter_size, stride=(1, 1),    # 1000
                         W=init.HeNormal(), b=None, nonlinearity=None, pad='same')
        net[name+'_1resid_norm'] = layers.BatchNormLayer(net[name+'_1resid'])
        net[name+'_1resid_active'] = layers.NonlinearityLayer(net[name+'_1resid_norm'], nonlinearity=nonlinearity)

        net[name+'_1resid_dropout'] = layers.DropoutLayer(net[name+'_1resid_active'], p=0.1)

        # bottleneck residual layer
        net[name+'_2resid'] = layers.Conv2DLayer(net[name+'_1resid_dropout'], num_filters=num_filters, filter_size=filter_size, stride=(1, 1),    # 1000
                         W=init.HeNormal(), b=None, nonlinearity=None, pad='same')
        net[name+'_2resid_norm'] = layers.BatchNormLayer(net[name+'_2resid'])

        # combine input with residuals
        net[name+'_residual'] = layers.ElemwiseSumLayer([net[last_layer], net[name+'_2resid_norm']])
        net[name+'_resid'] = layers.NonlinearityLayer(net[name+'_residual'], nonlinearity=nonlinearity)

        return net

    def residual_block2(net, last_layer, name, nonlinearity=nonlinearities.rectify):
        # original residual unit
        shape = layers.get_output_shape(net[last_layer])
        num_units = shape[1]

        net[name+'_1resid'] = layers.DenseLayer(net[last_layer], num_units=num_units, W=init.HeNormal(), b=None, nonlinearity=None)
        net[name+'_1resid_norm'] = layers.BatchNormLayer(net[name+'_1resid'])
        net[name+'_1resid_active'] = layers.NonlinearityLayer(net[name+'_1resid_norm'], nonlinearity=nonlinearity)

        net[name+'_1resid_dropout'] = layers.DropoutLayer(net[name+'_1resid_active'], p=0.1)

        # bottleneck residual layer
        net[name+'_2resid'] = layers.DenseLayer(net[name+'_1resid_dropout'], num_units=num_units, W=init.HeNormal(), b=None, nonlinearity=None)
        net[name+'_2resid_norm'] = layers.BatchNormLayer(net[name+'_2resid'])

        # combine input with residuals
        net[name+'_residual'] = layers.ElemwiseSumLayer([net[last_layer], net[name+'_2resid_norm']])
        net[name+'_resid'] = layers.NonlinearityLayer(net[name+'_residual'], nonlinearity=nonlinearity)

        return net

    
    # get model
    input_var = T.tensor4('inputs')
    target_var = T.dmatrix('targets')

    net = {}
    net['input'] = layers.InputLayer(input_var=input_var, shape=shape)
    net['conv1'] = layers.Conv2DLayer(net['input'], num_filters=32, filter_size=(3, 11), stride=(1, 1),    # 320
                                        W=init.HeNormal(), b=None, nonlinearity=None, pad='valid')
    net['conv1_norm'] = layers.BatchNormLayer(net['conv1'])
    net['conv1_active'] = layers.NonlinearityLayer(net['conv1_norm'], nonlinearity=nonlinearities.leaky_rectify)
    net['conv1_dropout1'] = layers.DropoutLayer(net['conv1_active'], p=0.1)
    net = residual_block(net, 'conv1_dropout1', 'conv1_2', filter_size=(1,5), nonlinearity=nonlinearities.leaky_rectify)
    net['conv1_pool'] = layers.MaxPool2DLayer(net['conv1_2_resid'], pool_size=(1, 32), stride=(1, 32)) # 13
    net['conv1_dropout'] = layers.DropoutLayer(net['conv1_pool'], p=0.1)

    net['dense1'] = layers.DenseLayer(net['conv1_dropout'], num_units=128, W=init.HeNormal(), 
                                     b=None, nonlinearity=None)
    net['dense1_norm'] = layers.BatchNormLayer(net['dense1'])    
    net['dense1_active'] = layers.NonlinearityLayer(net['dense1_norm'], nonlinearity=nonlinearities.leaky_rectify)
    net['dense1_dropout1'] = layers.DropoutLayer(net['dense1_active'], p=0.1)
    net = residual_block2(net, 'dense1_dropout1', 'dense1_2', nonlinearity=nonlinearities.leaky_rectify)
    net['dense1_dropout'] = layers.DropoutLayer(net['dense1_2_resid'], p=0.3)
    
    net['dense2'] = layers.DenseLayer(net['dense1_dropout'], num_units=num_labels, W=init.HeNormal(), 
                                     b=init.Constant(), nonlinearity=None)
    net['output'] = layers.NonlinearityLayer(net['dense2'], nonlinearity=nonlinearities.sigmoid)
    
    # optimization parameters
    optimization = {"objective": "binary",
                    "optimizer": "adam",
                    "learning_rate": 0.001, 
                    "l2": 1e-6
                    }

    return net, input_var, target_var, optimization

# build network
shape = (None, train[0].shape[1], train[0].shape[2], train[0].shape[3])
num_labels = train[1].shape[1]
net, input_var, target_var, optimization = build_model(shape, num_labels)

# build neural network class
nnmodel = NeuralNet(net, input_var, target_var)
#nnmodel.inspect_layers()

# set output file paths
resultspath = utils.make_directory(data_path, 'results')
resultspath = utils.make_directory(resultspath, 'deepomics')
output_name = '1d_version'
filepath = os.path.join(resultspath, output_name)
nntrainer = NeuralTrainer(nnmodel, optimization, save='best', filepath=filepath)

In [ ]:
# train model
nntrainer.set_best_parameters()
fit.train_minibatch(nntrainer, data={'train': train, 'valid': valid}, 
                              batch_size=100, num_epochs=20, patience=5, verbose=1)
# train model
nnmodel = NeuralNet(net, input_var, target_var)
nntrainer = NeuralTrainer(nnmodel, optimization, save='best', filepath=filepath)
nntrainer.set_best_parameters()
fit.train_minibatch(nntrainer, data={'train': train, 'valid': valid}, 
                              batch_size=500, num_epochs=20, patience=5, verbose=1)

nnmodel = NeuralNet(net, input_var, target_var)
nntrainer = NeuralTrainer(nnmodel, optimization, save='best', filepath=filepath)
nntrainer.set_best_parameters()
fit.train_minibatch(nntrainer, data={'train': train, 'valid': valid}, 
                              batch_size=1000, num_epochs=10, patience=5, verbose=1)

nnmodel = NeuralNet(net, input_var, target_var)
nntrainer = NeuralTrainer(nnmodel, optimization, save='best', filepath=filepath)
nntrainer.set_best_parameters()
fit.train_minibatch(nntrainer, data={'train': train, 'valid': valid}, 
                              batch_size=1500, num_epochs=10, patience=5, verbose=1)

nnmodel = NeuralNet(net, input_var, target_var)
nntrainer = NeuralTrainer(nnmodel, optimization, save='best', filepath=filepath)
nntrainer.set_best_parameters()
fit.train_minibatch(nntrainer, data={'train': train, 'valid': valid}, 
                              batch_size=2000, num_epochs=10, patience=5, verbose=1)


In [ ]:
nntrainer.set_best_parameters()

# test model
nntrainer.test_model(valid, batch_size=100, name="test")

In [ ]:
# check official validation error
auc = validation_results(nnmodel)
print('validation AUC: ' + str(auc))

# 1D residual model (modified)

In [ ]:
def build_model(shape, num_labels):
    def residual_block(net, last_layer, name, filter_size, nonlinearity=nonlinearities.rectify):
        # original residual unit
        shape = layers.get_output_shape(net[last_layer])
        num_filters = shape[1]

        net[name+'_1resid'] = layers.Conv2DLayer(net[last_layer], num_filters=num_filters, filter_size=filter_size, stride=(1, 1),    # 1000
                         W=init.HeNormal(), b=None, nonlinearity=None, pad='same')
        net[name+'_1resid_norm'] = layers.BatchNormLayer(net[name+'_1resid'])
        net[name+'_1resid_active'] = layers.NonlinearityLayer(net[name+'_1resid_norm'], nonlinearity=nonlinearity)

        net[name+'_1resid_dropout'] = layers.DropoutLayer(net[name+'_1resid_active'], p=0.1)

        # bottleneck residual layer
        net[name+'_2resid'] = layers.Conv2DLayer(net[name+'_1resid_dropout'], num_filters=num_filters, filter_size=filter_size, stride=(1, 1),    # 1000
                         W=init.HeNormal(), b=None, nonlinearity=None, pad='same')
        net[name+'_2resid_norm'] = layers.BatchNormLayer(net[name+'_2resid'])

        # combine input with residuals
        net[name+'_residual'] = layers.ElemwiseSumLayer([net[last_layer], net[name+'_2resid_norm']])
        net[name+'_resid'] = layers.NonlinearityLayer(net[name+'_residual'], nonlinearity=nonlinearity)

        return net

    
    def residual_block2(net, last_layer, name, nonlinearity=nonlinearities.rectify):
        # original residual unit
        shape = layers.get_output_shape(net[last_layer])
        num_units = shape[1]

        net[name+'_1resid'] = layers.DenseLayer(net[last_layer], num_units=num_units, W=init.HeNormal(), b=None, nonlinearity=None)
        net[name+'_1resid_norm'] = layers.BatchNormLayer(net[name+'_1resid'])
        net[name+'_1resid_active'] = layers.NonlinearityLayer(net[name+'_1resid_norm'], nonlinearity=nonlinearity)

        net[name+'_1resid_dropout'] = layers.DropoutLayer(net[name+'_1resid_active'], p=0.1)

        # bottleneck residual layer
        net[name+'_2resid'] = layers.DenseLayer(net[name+'_1resid_dropout'], num_units=num_units, W=init.HeNormal(), b=None, nonlinearity=None)
        net[name+'_2resid_norm'] = layers.BatchNormLayer(net[name+'_2resid'])

        # combine input with residuals
        net[name+'_residual'] = layers.ElemwiseSumLayer([net[last_layer], net[name+'_2resid_norm']])
        net[name+'_resid'] = layers.NonlinearityLayer(net[name+'_residual'], nonlinearity=nonlinearity)

        return net


    # get model
    input_var = T.tensor4('inputs')
    target_var = T.dmatrix('targets')

    net = {}
    net['input'] = layers.InputLayer(input_var=input_var, shape=shape) # 330
    net['conv1'] = layers.Conv2DLayer(net['input'], num_filters=32, filter_size=(3, 7), stride=(1, 1), # 324
                                        W=init.HeNormal(), b=None, nonlinearity=None, pad='valid')
    net['conv1_norm'] = layers.BatchNormLayer(net['conv1'])
    net['conv1_active'] = layers.NonlinearityLayer(net['conv1_norm'], nonlinearity=nonlinearities.leaky_rectify)
    net['conv1_dropout1'] = layers.DropoutLayer(net['conv1_active'], p=0.1)
    net = residual_block(net, 'conv1_dropout1', 'conv1_2', filter_size=(1,5), nonlinearity=nonlinearities.leaky_rectify)
    net['conv1_pool'] = layers.MaxPool2DLayer(net['conv1_2_resid'], pool_size=(1, 12), stride=(1, 12)) # 27
    net['conv1_dropout'] = layers.DropoutLayer(net['conv1_pool'], p=0.1)

    net['conv2'] = layers.Conv2DLayer(net['conv1_dropout'], num_filters=64, filter_size=(1, 7), stride=(1, 1), # 21
                                        W=init.HeNormal(), b=None, nonlinearity=None, pad='valid')
    net['conv2_norm'] = layers.BatchNormLayer(net['conv2'])
    net['conv2_active'] = layers.NonlinearityLayer(net['conv2_norm'], nonlinearity=nonlinearities.leaky_rectify)
    net['conv2_dropout1'] = layers.DropoutLayer(net['conv2_active'], p=0.1)
    net = residual_block(net, 'conv2_dropout1', 'conv2_2', filter_size=(1,5), nonlinearity=nonlinearities.leaky_rectify)
    net['conv2_pool'] = layers.MaxPool2DLayer(net['conv2_2_resid'], pool_size=(1, 7), stride=(1, 7)) # 3
    net['conv2_dropout'] = layers.DropoutLayer(net['conv2_pool'], p=0.1)

    net['dense1'] = layers.DenseLayer(net['conv2_dropout'], num_units=128, W=init.HeNormal(), 
                                     b=None, nonlinearity=None)
    net['dense1_norm'] = layers.BatchNormLayer(net['dense1'])    
    net['dense1_active'] = layers.NonlinearityLayer(net['dense1_norm'], nonlinearity=nonlinearities.leaky_rectify)
    net['dense1_dropout1'] = layers.DropoutLayer(net['dense1_active'], p=0.1)
    net = residual_block2(net, 'dense1_dropout1', 'dense1_2', nonlinearity=nonlinearities.leaky_rectify)
    net['dense1_dropout'] = layers.DropoutLayer(net['dense1_2_resid'], p=0.3)
    
    net['dense2'] = layers.DenseLayer(net['dense1_dropout'], num_units=num_labels, W=init.HeNormal(), 
                                     b=init.Constant(), nonlinearity=None)
    net['output'] = layers.NonlinearityLayer(net['dense2'], nonlinearity=nonlinearities.sigmoid)
    
    # optimization parameters
    optimization = {"objective": "binary",
                    "optimizer": "adam",
                    "learning_rate": 0.001, 
                    "l2": 1e-6
                    }

    return net, input_var, target_var, optimization

# build network
shape = (None, train[0].shape[1], train[0].shape[2], train[0].shape[3])
num_labels = train[1].shape[1]
net, input_var, target_var, optimization = build_model(shape, num_labels)

# build neural network class
nnmodel = NeuralNet(net, input_var, target_var)
#nnmodel.inspect_layers()

# set output file paths
resultspath = utils.make_directory(data_path, 'results')
resultspath = utils.make_directory(resultspath, 'deepomics')
output_name = '1D_model_resid'
filepath = os.path.join(resultspath, output_name)
nntrainer = NeuralTrainer(nnmodel, optimization, save='best', filepath=filepath)

In [ ]:
# train model
#nntrainer.set_best_parameters()
fit.train_minibatch(nntrainer, data={'train': train, 'valid': valid}, 
                              batch_size=100, num_epochs=50, patience=10, verbose=1)
# train model
nnmodel = NeuralNet(net, input_var, target_var)
nntrainer = NeuralTrainer(nnmodel, optimization, save='best', filepath=filepath)
nntrainer.set_best_parameters()
fit.train_minibatch(nntrainer, data={'train': train, 'valid': valid}, 
                              batch_size=500, num_epochs=50, patience=10, verbose=1)

nnmodel = NeuralNet(net, input_var, target_var)
nntrainer = NeuralTrainer(nnmodel, optimization, save='best', filepath=filepath)
nntrainer.set_best_parameters()
fit.train_minibatch(nntrainer, data={'train': train, 'valid': valid}, 
                              batch_size=1000, num_epochs=50, patience=10, verbose=1)

nnmodel = NeuralNet(net, input_var, target_var)
nntrainer = NeuralTrainer(nnmodel, optimization, save='best', filepath=filepath)
nntrainer.set_best_parameters()
fit.train_minibatch(nntrainer, data={'train': train, 'valid': valid}, 
                              batch_size=1500, num_epochs=50, patience=10, verbose=1)

nnmodel = NeuralNet(net, input_var, target_var)
nntrainer = NeuralTrainer(nnmodel, optimization, save='best', filepath=filepath)
nntrainer.set_best_parameters()
fit.train_minibatch(nntrainer, data={'train': train, 'valid': valid}, 
                              batch_size=2000, num_epochs=50, patience=10, verbose=1)


In [ ]:
nntrainer.set_best_parameters()

# test model
nntrainer.test_model(valid, batch_size=100, name="test")

In [ ]:
# check official validation error
auc = validation_results(nnmodel)
print('validation AUC: ' + str(auc))